In [1]:
# run this if you want to run the example from the examples folder without installing the package
import sys
sys.path.insert(0, "../")

In [2]:
import grpc
from osmosis_proto.proto.cosmos.bank.v1beta1.query_pb2 import QueryBalanceRequest

In [5]:
# configure grpc channel

ssl_creds = grpc.ssl_channel_credentials()
grpc_endpoint = "grpc-test.osmosis.zone:443" # testnet

channel = grpc.secure_channel(grpc_endpoint, ssl_creds)

# prepare request
req = QueryBalanceRequest(address="osmo1fl48vsnmsdzcv85q5d2q4z5ajdha8yu3aq6l09")

# send request
resp = channel.unary_unary("/cosmos.bank.v1beta1.Query/Balance")(req.SerializeToString())

channel.close()

E0324 13:09:12.838603582   15851 ssl_transport_security.cc:1501]       Handshake failed with fatal error SSL_ERROR_SSL: error:100000f7:SSL routines:OPENSSL_internal:WRONG_VERSION_NUMBER.


_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: ipv6:%5B64:ff9b::8fc6:8b21%5D:443: Network is unreachable"
	debug_error_string = "UNKNOWN:failed to connect to all addresses; last error: UNKNOWN: ipv6:%5B64:ff9b::8fc6:8b21%5D:443: Network is unreachable {grpc_status:14, created_time:"2023-03-24T13:09:12.839171595+01:00"}"
>

E0324 13:09:22.167346907   15872 ssl_transport_security.cc:1501]       Handshake failed with fatal error SSL_ERROR_SSL: error:100000f7:SSL routines:OPENSSL_internal:WRONG_VERSION_NUMBER.
E0324 13:09:32.167523383   15906 ssl_transport_security.cc:1501]       Handshake failed with fatal error SSL_ERROR_SSL: error:100000f7:SSL routines:OPENSSL_internal:WRONG_VERSION_NUMBER.
E0324 13:09:42.167512389   15906 ssl_transport_security.cc:1501]       Handshake failed with fatal error SSL_ERROR_SSL: error:100000f7:SSL routines:OPENSSL_internal:WRONG_VERSION_NUMBER.
E0324 13:09:52.167489266   15872 ssl_transport_security.cc:1501]       Handshake failed with fatal error SSL_ERROR_SSL: error:100000f7:SSL routines:OPENSSL_internal:WRONG_VERSION_NUMBER.
E0324 13:10:02.167510356   15872 ssl_transport_security.cc:1501]       Handshake failed with fatal error SSL_ERROR_SSL: error:100000f7:SSL routines:OPENSSL_internal:WRONG_VERSION_NUMBER.
E0324 13:10:12.167700509   15906 ssl_transport_security.cc:1501] 